In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

// TODO : Find out how to restart kernel

In [5]:
%%plantuml 

@startuml
Alice -> Bob: Authentication Request
Bob --> Alice: Authentication Response
@enduml 

UsageError: Cell magic `%%plantuml` not found.


In [41]:
class Microwave:
    def __init__(self):
        pass

    def heat_up_food(self):
        print("Food is being microwaved")

    def boil_water(self):
        print("Water is being boiled")

    def ruin_cake(self):
        print("Cake is being ruined... ah come on...why even lol")

class Dishwasher:
    def __init__(self):
        pass

    def wash_dishes(self):
        print("Dishwasher starting")

    def leave_spots(self):
        print("Dishes have spots, have fun being embarassed in front of guests")

class Kitchen ():
    def __init__(self):
        self.microwave = Microwave()
        self.dishwasher = Dishwasher()
        # how to unit test not callable?
        self.microwave_methods = [f for f in dir(self.microwave) if not f.startswith('_') and callable(getattr(self.microwave, f))]
        # checking for callable overhead
        self.dishwasher_methods = [f for f in dir(self.dishwasher) if not f.startswith('_') and callable(getattr(self.dishwasher, f))]

    def __getattr__(self, func):
        def method(*args):
            if func in self.microwave_methods:
                return getattr(self.microwave, func)(*args)
            if func in self.dishwasher_methods:
                return getattr(self.dishwasher, func)(*args)
            else:
                raise AttributeError(f"Method {func} not found")
        return method


In [42]:
kitchen = Kitchen()
kitchen.wash_dishes()
kitchen.boil_water()
kitchen.ruin_cake()
kitchen.leave_spots()

Dishwasher starting
Water is being boiled
Cake is being ruined... ah come on...why even lol
Dishes have spots, have fun being embarassed in front of guests
